# Application - construire une carte interactive (type google map)

**Mini-projet guidé**: afficher dans une carte interactive centrée sur Tours toutes les gares de *voyageurs* d'indre-et-loire avec leur nom.

Les ingrédients de ce mini-projet:

1. Récupérer un fichier au format CSV contenant l'ensemble des gares de france,
2. Traitement des données avec Python:
    - Adapter les données pour Python,
    - Sélectionner les objets (gares) pertinents,
    - Ne conserver que les descripteurs utiles,
    - Transformer les données textes en données compatibles avec l'outil utilisé dans l'étape 3;
3. Utilisation de la bibliothèque *ipyleaflet* pour afficher et manipuler une carte interactive.

## ipyleaflet

Cette bibliothèque sert à afficher des cartes interactives dans un notebook.

Elle est basée sur la bibliothèque (en javascript) [leaflet](https://leafletjs.com/)

Pour l'utiliser, on commence par importer (elle est installée sur le hub) deux constructeurs `Map` (carte) et `Marker` (pour créer des marques sur la cartes)

In [ ]:
from ipyleaflet import Map, Marker

Commençons par construire la carte

In [ ]:
# Coordonnées de bordeaux pour centrer la carte
bordeaux = (44.833333,-0.566667)

#L'objet carte
carte = Map(center=bordeaux, zoom=10)
display(carte) # display est une fonction spéciale des notebook

Ajoutons un petit marqueur

In [ ]:
marker = Marker(location=bordeaux, title='bordeau', draggable=False)
carte.add_layer(marker) # regarder de nouveau la carte.

Simple non? Si vous souhaitez en savoir davantage chercher dans la [documentation de ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/)

## Récupération des données au format CSV

Rendez vous sur le site [data.sncf.com](https://data.sncf.com) et chercher la liste des gares au format csv.

Une fois le fichier récupéré, enregistrer le dans le même dossier que ce notebook et renommer le `gares.csv`

### Lire le fichier avec Python

Pour lire un fichier texte avec Python, on utilise la syntaxe:

In [ ]:
with open('gares.csv') as f_csv:
    csv = f_csv.read()

# pour afficher 500 premiers caractères de la chaîne.
print(csv[:500])

Cela fonctionne très bien ici mais le fichier contient (**premier hic**) un caractère spécial `\uefeff` (invisible); *enlever le print pour le voir*.

Pour l'éliminer (il risque de gêner autrement), un petit slice suffit.

In [ ]:
csv = csv.strip() # éliminer les blancs (il y a un saut de ligne tout à la fin du fichier! souvent le cas)
csv = csv[1:] # élimine le caractère zarbi dont je vous parlais...

Il ne reste plus qu'à exploiter comme vu précédemment la chaine `csv`...

Mais il y a un **deuxième hic** (encore? et oui c'est souvent comme ça avec les exemples réels) ... **l'avez-vous remarqué?**

Le séparateur n'est pas la virgule `,` mais le point-virgule `;`: c'est souvent le cas pour les fichiers csv français car la virgule est utilisée comme séparateur décimal.

Il va donc falloir adapter le code qui va nous permettre de tranformer cette chaîne en un tableau de données (sous forme de n-uplets nommés)

### À vous de jouer!

Compléter la fonction suivante pour effectuer la conversion.

In [ ]:
def csv_vers_tableau(csv_str, sep=','): # valeur par défaut
    lignes = csv_str.split(___)
    descripteurs = tuple(lignes[___].split(sep))
    liste_objs = [tuple(___.split(___)) for l in lignes[1:]]
    
    objets = []
    for o in ____:
        objet = {}
        for d, v in ___(descripteurs, o):
            objet[d] = ____
        objets.append(____)
    return objets

In [ ]:
def csv_vers_tableau(csv_str, sep=','): # valeur par défaut
    lignes = csv_str.split('\n')
    descripteurs = tuple(lignes[0].split(sep))
    liste_objs = [tuple(l.split(sep)) for l in lignes[1:]]
    
    objets = []
    for o in liste_objs:
        objet = {}
        if len(descripteurs) != len(o):
            print(descripteurs)
            print(o)
        for d, v in zip(descripteurs, o):
            objet[d] = v
        objets.append(objet)
    return objets

In [ ]:
# vérifier que tout se passe bien pour les gares
tableau_gares = csv_vers_tableau(csv, ';')
tableau_gares[:2]

Regarder attentivement comment les gares sont décrites.

Votre **objectif** est d'afficher toutes les gares de voyageurs d'Indre-et-Loire sur une carte interactive.

*Conseils*: Il va falloir:
- sélectionner les bonnes gares (**sélection**),
- ne conserver que les informations pertinentes (**projection**),
- effectuer un traitement de certaines chaînes de caractères (**pré-traitement**),
- enfin, utiliser effectivement la table en lien avec la carte interactive fournie par **ipyleaflet**.

# À vous de jouer!

In [ ]:
# 1. Sélectionner les bonnes gares: utiliser le descripteur 'DEPARTEMENT' avec la valeur 'INDRE-ET-LOIRE'
selection1 = [ gare for gare in tableau_gares if gare['DEPARTEMENT'] == 'INDRE-ET-LOIRE' ]
len(selection1)

In [ ]:
# 2. On veut seulement les gares de voyageurs
selection2 = [ gare for gare in selection1 if gare['VOYAGEURS'] == 'O' ]
len(selection2)

In [ ]:
# 3. projection: on veut juste les colonnes 'LIBELLE' et 'C_GEO'
projection = [ {d: v for d, v in g.items() if d in ['LIBELLE', 'C_GEO']} for g in selection2]
projection[:1]

In [ ]:
# 4. transformer la chaîne des coordonnées en tuple (float, float)
def coord(ch):
    x, y = ch.split(',')
    return (float(x), float(y))

gares = [ {d: coord(v) if d == 'C_GEO' else v for d, v in o.items()} for o in projection ]
gares[:2]

In [ ]:
# 5. constuire la carte et y ajouter un marqueur par gare
from ipyleaflet import Map, Marker

# recherche sur internet
tours = (47.383331, 0.68333)

# L'objet carte
carte = Map(center=tours, zoom=10)
display(carte) # display est une fonction spéciale des notebook

# Mettre les gares.
for gare in gares:
    marker = Marker(location=gare['C_GEO'], title=gare['LIBELLE'], draggable=False)
    carte.add_layer(marker)